# IDEAS
1. If the custom reading speed for a book is set, take that for calculating the number of days or take the generic one asked during program execution
2. If the custom number of times to read for a book is set, take that for calculating the number of days or take the generic one asked during program execution
3. Do not set the target date to complete - let it tell you what the target completion date is for your selections
4. Option to reselect the reading speed and run the function again to tell the target date

# TODOS
1. **DONE**: Remove the need to populate the excel file with dates. Add the dates from today or a custom date.
2. **DONE**: Take the input of reading speed while running the program
3. Add the column of reading speed in excel to set custom reading speed for each book
4. Add the column of number of times to read a book - prompt for an input while starting program execution

In [6]:
import pandas as pd
import numpy as np
import datetime

In [7]:
weekday_reading_speed = int(input("Enter the reading speed on weekday:"))
weekend_reading_speed = int(input("Enter the reading speed on weekend:"))

In [8]:
today = datetime.date.today()
print(today.isoweekday())

2


In [9]:
tomorrow = today + datetime.timedelta(days=1)
print(tomorrow)

2022-08-03


In [33]:
#todo abstract the implementation of the while with an iterator for the same
list_of_dates = []
while tomorrow < datetime.date(2023, 1, 1):
    list_of_dates.append(tomorrow)
    tomorrow += datetime.timedelta(days=1)
print(len(list_of_dates))

0


In [11]:
# days = pd.read_excel('Days.xlsx', sheet_name = 'Days', parse_dates=['Dates'])
days = pd.DataFrame(list_of_dates, columns=["Date"])

In [12]:
days.head()

,Date
0,2022-08-02
1,2022-08-03
2,2022-08-04
3,2022-08-05
4,2022-08-06


In [13]:
days.dtypes

Date    object
dtype: object

In [14]:
days['Date'] = pd.to_datetime(days['Date'])
days.dtypes

Date    datetime64[ns]
dtype: object

In [25]:
books = pd.read_excel('Days.xlsx', sheet_name='Books')
books.head()

,Name,Class,Pages
0,Macroeconomic Policy Environment - Shyamal Roy,NaN,126
1,Indian Economic Development,11.0,198
2,Indian Art and Culture - Nitin Singhania,NaN,532


In [121]:
workdays = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
weekends = ['Saturday', 'Sunday']

In [21]:
def pages(day: datetime.date):
    day_of_week = day.isoweekday()
    if day_of_week < 6:
        return weekday_reading_speed
    return weekend_reading_speed

In [22]:
# pages(days.iloc[[0]].Weekdays[0])
pages(days.iat[0,0])

40

In [23]:
days.dtypes

Date      datetime64[ns]
Target             int64
Book              object
Class              int64
dtype: object

In [18]:
days['Target'] = 0
days['Book'] = ''
days['Class'] = 0
days.head()

,Date,Target,Book,Class
0,2022-08-02,0,,0
1,2022-08-03,0,,0
2,2022-08-04,0,,0
3,2022-08-05,0,,0
4,2022-08-06,0,,0


In [26]:
currDay = 0
targetPerDay = []
bookPerDay = []
classPerDay = []
for bookNumber in books.index:
    book = books.loc[bookNumber]
    pagesRemaining = book.Pages
    # print(type(pagesRemaining))
    # print(pagesRemaining)
    # print(pagesRemaining - 20)
    # break
    while pagesRemaining > 0 and currDay <= len(days):
        day = days.loc[currDay]
        targetPerDay.append(pages(day.Date) if pages(day.Date) < pagesRemaining else pagesRemaining)
        bookPerDay.append(book.Name)
        classPerDay.append(book.Class)
        pagesRemaining -= targetPerDay[-1]
        currDay += 1
    if pagesRemaining != 0:
        print('Cannot allocate all books. Increase the number of pages covered in each day')
        break
else:
    print('Allocated Successfully!')
    print('Total Pages =', sum(targetPerDay))
    print('Total Days Required at Current Speed =', len(targetPerDay))
    
    

Allocated Successfully!
Total Pages = 856
Total Days Required at Current Speed = 22


In [27]:
if len(targetPerDay) < len(days):
    diff = len(days) - len(targetPerDay)
    for item in [targetPerDay, bookPerDay, classPerDay]:
        item.extend([None]*diff)

In [28]:
print(len(targetPerDay), len(bookPerDay), len(classPerDay))

152 152 152


In [29]:
days.Target = targetPerDay
days.Book = bookPerDay
days.Class = classPerDay
days.head()

,Date,Target,Book,Class
0,2022-08-02,40.0,Macroeconomic Policy Environment - Shyamal Roy,NaN
1,2022-08-03,40.0,Macroeconomic Policy Environment - Shyamal Roy,NaN
2,2022-08-04,40.0,Macroeconomic Policy Environment - Shyamal Roy,NaN
3,2022-08-05,6.0,Macroeconomic Policy Environment - Shyamal Roy,NaN
4,2022-08-06,50.0,Indian Economic Development,11.0


In [31]:
days.Date = days.Date.apply(lambda x: x.strftime('%d-%m-%Y'))

In [32]:
days.head()

,Date,Target,Book,Class
0,02-08-2022,40.0,Macroeconomic Policy Environment - Shyamal Roy,NaN
1,03-08-2022,40.0,Macroeconomic Policy Environment - Shyamal Roy,NaN
2,04-08-2022,40.0,Macroeconomic Policy Environment - Shyamal Roy,NaN
3,05-08-2022,6.0,Macroeconomic Policy Environment - Shyamal Roy,NaN
4,06-08-2022,50.0,Indian Economic Development,11.0


In [141]:
with pd.ExcelWriter('Days.xlsx', mode='a', if_sheet_exists='replace') as writer:
    days.to_excel(writer, sheet_name='Plan')